In [2]:
import torch
import torch.nn.functional as F
import numpy as np



import plotly.graph_objects as go 
import plotly.express as px
from plotly  import subplots

In [3]:
from typing import Tuple, List, Optional
ImageType = torch.Tensor
TensorType = torch.Tensor

Invariance to local translation can be a very useful property if we care more about whether some feature is present than exactly where it is. For example, when determining whether an image contains a face, we need not know the location of the eyes with pixel-perfect accuracy, we just need to know that there is an eye on the left side of the face and an eye on the right side of the face.

— Page 342, Deep Learning, 2016.


[אתר מדהים שממחיש את בעזרת קונבלוציה ניתן לעבד תמונות](https://setosa.io/ev/image-kernels/)

Our goal is to find out how gradient is propagating backwards in a convolutional layer. The forward pass is defined like this:

The input consists of N data points, each with C channels, height H and width W. We convolve each input with F different filters, where each filter spans all C channels and has height HH and width WW.

Input:

    x: Input data of shape (N, C, H, W)
    w: Filter weights of shape (F, C, HH, WW)
    b: Biases, of shape (F,)
    conv_param: A dictionary with the following keys:
    ‘stride’: The number of pixels between adjacent receptive fields in the horizontal and vertical directions.
    ‘pad’: The number of pixels that will be used to zero-pad the input.

During padding, ‘pad’ zeros should be placed symmetrically (i.e equally on both sides) along the height and width axes of the input.

Returns a tuple of:

    out: Output data, of shape (N, F, H’, W’) where H’ and W’ are given by

H’ = 1 + (H + 2 * pad — HH) / stride

W’ = 1 + (W + 2 * pad — WW) / stride

    cache: (x, w, b, conv_param)

<div dir="rtl" lang="he" xml:lang="he">

## שימוש בפונקציית קונבלוציה
פונקציית קונבלוציה מאפשרת לנו למצוא תבניות בתוך התמונות, בהדגמה להלן נראה איך פונקציית קונבלוציה יכולה למצוא תבנית מסויימת בתוך תמונה

In [11]:
im = torch.zeros(1, 1, 100, 100)
rand_index= torch.randint(0, 100, (10,))
im[:, :, :, rand_index] = 1
rand_index= torch.randint(0, 100, (10,))
im[:, :,rand_index,:] = 1
px.imshow(im[0, 0, :, :])


In [12]:
pattern = torch.zeros(4, 1, 3, 3)
pattern[0, 0, :, 1] = 1
pattern[1, 0, 1, :] = 1
pattern[3, 0, :, [0,2]] = -1
pattern[3, 0, :, 1] = 1
grid = torch.cat((pattern[0,0], pattern[1,0], pattern[2,0], pattern[3,0]))
# go.Figure(data=[go.Heatmap(z=grid)])
print(grid)

tensor([[ 0.,  1.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  0.],
        [ 1.,  1.,  1.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [-1.,  1., -1.],
        [-1.,  1., -1.],
        [-1.,  1., -1.]])


In [20]:
pattern = torch.zeros(4, 1, 3, 3)
pattern[0, 0, :, 1] = 1
pattern[1, 0, 1, :] = 1
pattern[3, 0, :, [0, 2]] = -1
pattern[3, 0, :, 1] = 1
grid = torch.cat((pattern[0, 0], pattern[1, 0], pattern[2, 0], pattern[3, 0]), 1)
go.Figure(data=[go.Heatmap(z=grid)]).update_layout(yaxis_scaleanchor="x")
# print(grid)


<div dir="rtl" lang="he" xml:lang="he">

נוסיף רעש לתמונה ונראה אם הפילטרים שלנו _kernels_
יצליחו לזהות את הצורות בתוך התמונה

In [21]:
im  = im+torch.empty_like(im).uniform_(-1.,1.)
px.imshow(im[0,0,:,:])

In [23]:
feature_map = torch.conv2d(im, pattern)
fig = subplots.make_subplots(1, 3,shared_xaxes="all").update_layout(yaxis_scaleanchor="x")
fig.add_traces(
    data=[
        go.Heatmap(z=feature_map[0, 0, :, :], coloraxis="coloraxis"),
        go.Heatmap(z=feature_map[0, 1, :, :], coloraxis="coloraxis"),
        go.Heatmap(z=feature_map[0, 3, :, :], coloraxis="coloraxis"),
    ],
    rows=[1, 1,1],
    cols=[1, 2,3],
)
fig.show()


<div dir="rtl" lang="he" xml:lang="he">
נעשה כעת את השכבה עם פונקציה אקטיבציה, כך נוכל לקבל מידע יותר ברור איפה נמצאים הקווים

In [27]:
cov_im = torch.conv2d(im, pattern)
feature_map_relu = F.relu(cov_im)
feature_map_leaky_relu = F.leaky_relu(cov_im-1)
feature_map_sigmoid = F.sigmoid(cov_im)
feature_map_tanh = F.tanh(cov_im)
fig = subplots.make_subplots(2, 2, shared_xaxes="all")
fig.add_traces(
    data=[
        go.Heatmap(
            z=feature_map_relu[0, 3, :, :],
            coloraxis="coloraxis",
            name="relu"
        ),
        go.Heatmap(
            z=feature_map_leaky_relu[0, 3, :, :],
            coloraxis="coloraxis",
            name="leaky_relu"
        ),
        go.Heatmap(
            z=feature_map_sigmoid[0, 3, :, :],
            coloraxis="coloraxis",
            name="sigmoid"
        ),
        go.Heatmap(z=feature_map_tanh[0, 3, :, :], coloraxis="coloraxis",name="tanh"),
    ],
    rows=[1, 1, 2, 2],
    cols=[1, 2, 1, 2],
)
fig.update_layout(yaxis_scaleanchor="x")
fig.show()


<div dir="rtl" lang="he" xml:lang="he">

## זהוי התבנית בתוך תמונה מורעשת
ניצור תמונה עם תבנית כל שהיא במקומות אקראיים, ונראה איך פונקציית הקונבולוציה יכולה למצוא את המקום שלהם 

התמונות העוברת לפונקציית הקונבלוציה הם תמיד מהצורה הבאה:
`מספר נקודות`X`מספר ערוצים`X`גובה`X`רוחב` 

 במה שנעשה כעת נשתמש רק בערוץ אחד ובנקודה 
 לכן המימד של התמונה שלנו הוא `1,1,100,100`
 

<div dir="rtl" lang="he" xml:lang="he">

## מפת תכונות _feature map_
פונקציית הקונבולוציה מוציאה מפה פיצ'רים , לאחר הפעלת הפילטר.אחד השימושים הוא למצוא תבניות בתמונה, במקרה שלנו ננסה למצוא תבנית ספציפית שנמצאת בתמונה

In [ ]:
conve_layer = torch.nn.Conv2d(1,1,3,padding=(1,1),bias=False)
optimizer = torch.optim.SGD(conve_layer.parameters(),0.01,momentum=0.01)
loss_fucnction = torch.nn.MSELoss()
image = torch.rand(1,1,50,50)
target_filter = conve_layer.weight.detach()
out_f = torch.conv2d(image, target_filter,padding=(1,1))
out_layer = conve_layer(image)
fig,axis = plt.subplots(1,4,figsize=(20,5))
axis[0].imshow(image[0,0,:,:])
axis[1].imshow(out_f[0,0,:,:])
axis[2].imshow(out_layer.detach()[0,0,:,:])
axis[3].imshow((out_layer.detach()-out_f)[0,0,:,:])


image = torch.rand(1,1,50,50)
target_filter = torch.rand(1,1,3,3)
target = torch.conv2d(image, target_filter,padding=(1,1))
plt.imshow(target_filter[0,0,:,:])
loss_list = []
fig, (axis_width, axis_loss) = plt.subplots(1, 2, figsize=(10, 5))
for i in range(1000):
    out_layer = conve_layer(image)
    diff = loss_fucnction(target, out_layer)
    optimizer.zero_grad()
    diff.backward()
    optimizer.step()
    loss_list.append(diff.item())
    if i % 100 == 0:
        fig, (axis_width, axis_loss, axis_diff, axis_target_filter) = plt.subplots(
            1, 4, figsize=(15, 5)
        )
        axis_loss.plot(range(len(loss_list)), loss_list)
        annotated_heatmaps(conve_layer.weight.detach()[0, 0, :, :], axis_width,vmin=0, vmax=1)
        annotated_heatmaps(
            (conve_layer.weight.detach() - target_filter)[0, 0, :, :], axis_diff,vmin=0, vmax=1
        )
        annotated_heatmaps(target_filter[0, 0, :, :], axis_target_filter,vmin=0, vmax=1)
